# Let's Evaluate Tiny MoE 3





### Configure the machine and install packages


In [1]:
from typing import Literal

import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../alpha-448101-282bc1b884cd.json"
#os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=8'

import jax

platform : Literal["darwin", "colab", "cuda", "tpu"] = "darwin"

try:
    import google.colab
    platform = "colab"
except ImportError:
    devices = jax.devices()
    if any(d.platform == "gpu" for d in devices):
        platform = "cuda"
    if any(d.platform == "tpu" for d in devices):
        platform = "tpu"

print(f"Running on {platform}")

if platform == "colab":
    !git clone https://github.com/novastar53/jaxpt
    !cd jaxpt && git checkout main && git pull
    !pip install tiktoken datasets --quiet
    !pip uninstall -y tensorflow

Running on darwin


In [2]:
from pathlib import Path
import sys

if platform == "colab":
    jaxpt_dir = str(Path().absolute() / "jaxpt" / "src" )
else:
    jaxpt_dir = str(Path().absolute().parent / "src" )


sys.path.append(jaxpt_dir)
print(jaxpt_dir)

/Users/vikram/dev/jaxpt/src


In [3]:
import os


import warnings

import jax
from jax.sharding import Mesh, PartitionSpec, NamedSharding
import flax
import jax.numpy as jnp
import numpy as np

# Hardware setup
print("JAX version:", jax.__version__)
print("Flax version", flax.__version__)

devices = jax.devices()
num_devices = len(devices)
print("Available devices:", num_devices)

requested_device = "gpu"

jax.config.update("jax_platform_name", requested_device) # Make sure we're using the GPU

device = jax.default_backend()
if device != requested_device:
    warnings.warn(f"not using {requested_device}. Using {device}")
else:
    print(f"using {device}")


#####################################
##        jax.lax matmul presets   ##
#####################################
## 'ANY_F8_ANY_F8_F32',
## 'ANY_F8_ANY_F8_F32_FAST_ACCUM'
## 'ANY_F8_ANY_F8_ANY'
## 'ANY_F8_ANY_F8_ANY_FAST_ACCUM'
## 'F16_F16_F16'
## 'F16_F16_F32'
## 'BF16_BF16_BF16'
## 'BF16_BF16_F32'
## 'BF16_BF16_F32_X3'
## 'BF16_BF16_F32_X6'
## 'TF32_TF32_F32'
## 'TF32_TF32_F32_X3'
## 'F32_F32_F32'
## 'F64_F64_F64'
#####################################

#jax.config.update("jax_default_matmul_precision", "BF16_BF16_F32") # Set the default precision for matrix multiplication

#jax.config.update("jax_enable_x64", True) # Make sure the highest precision is enabled in case we need
#os.environ["NVIDIA_TF32_OVERRIDE"] = "1"
#os.environ["JAX_ENABLE_X64"] = "False"

if device == "tpu":
    def list_tpu_memory():
        devices = jax.devices()
        for device in devices:
            if 'TPU' in str(device.device_kind):
                print(f"Device: {device}, Memory: {device.memory_stats()['bytes_limit']/(1024*1024)},  Used: {device.memory_stats()['bytes_in_use']/(1024*1024)}")

    list_tpu_memory()

# Set up device mesh


mesh = jax.sharding.Mesh(jax.devices(), ["devices"])

# Test the device
A = jnp.array(np.random.normal(size=(4096, 4096)), dtype=jnp.float32) # Make sure matmul is fast
%timeit (A@A).block_until_ready()

JAX version: 0.7.0
Flax version 0.11.1
Available devices: 1


/var/folders/x4/85_9sn3d1ng9q48ff6t3fw340000gn/T/ipykernel_60134/1464138363.py:26: UserWarning: not using gpu. Using cpu
  warnings.warn(f"not using {requested_device}. Using {device}")


283 ms ± 89.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
from flax import nnx

from jaxpt.models import Tiny_MoE_3_Config, Tiny_MoE_3
from jaxpt.utils import count_params, create_sharded_model
from jaxpt.checkpointers import load_checkpoint_from_gcloud

import tiktoken
from transformers import AutoTokenizer

key = jax.random.PRNGKey(1337)
rngs = nnx.Rngs(key)
config = Tiny_MoE_3_Config(
                     name="Tiny_MoE_3",
                     dtype=jnp.bfloat16, \
                     vocab_size=49152,
                     n_layer=2,
                     block_size=2048,
                     n_head=12,
                     n_kv_head=4,
                     n_embed=672,
                     n_mlp_hidden=2048,
                     moe_bias=False,
                     mlp_bias=False,
                     attention_bias=False,
                     ln_epsilon = 1e-5,
                     sdpa_implementation="cudnn" if device=="gpu" else "xla")
nnx.display(config)
with mesh:
    m = create_sharded_model(Tiny_MoE_3, config, rngs)
    #m = load_checkpoint_from_gcloud(Tiny_MoE_2, config, Path().absolute().parent , "alpha_training_runs", "run_20250827_hart_narendra", 80000, rngs)
    #m = load_checkpoint(Tiny_MoE, output_dir, config, "run_20250716_htlpzj", 180000, rngs)
    #m = load_checkpoint(Tiny_MoE, output_dir, config, "run_20250716_htlpzj", 381469, rngs)

    graphdef, rngstate, state = nnx.split(m, nnx.RngState, ...)
    total_params = count_params(m)
    moe_params = count_params(m, "experts")
    print(f"Parameter Count: {total_params:,}")
    print(f"MOE (Sharded) Parameter Count: {moe_params:,}")
    replicated_params = total_params - moe_params
    print(f"Replicated Parameter Count: {replicated_params:,}")
    active_params = replicated_params + ( 2 / 8 ) * moe_params
    print(f"Active Parameter Count: {active_params:,}")
    fraction_of_active_params = active_params / total_params
    print(f"% Active Parameters: {fraction_of_active_params*100:0.2f} %")

    nnx.display(state)

Parameter Count: 73,977,120
MOE (Sharded) Parameter Count: 33,030,144
Replicated Parameter Count: 40,946,976
Active Parameter Count: 49,204,512.0
% Active Parameters: 66.51 %


In [5]:
from functools import partial
from transformers import AutoTokenizer
#from jaxpt.infer import generate_completion_slow
import jax


@nnx.jit
def _generate_step(m, x, attn_mask, temperature, key):
    logits = m(x, mask=attn_mask)
    logits = logits / temperature
    x_new = logits[:, -1, :]
    top_k_vals, top_k_indices = jax.lax.top_k(x_new, 50)
    key, subkey = jax.random.split(key)
    top_k_logit_idxs = jax.random.categorical(
        subkey, top_k_vals
    )
    top_k_logit_idxs = top_k_logit_idxs[..., None]  # expand dims
    sample_idxs = jnp.take_along_axis(
        top_k_indices, top_k_logit_idxs, axis=-1
    )
    return sample_idxs


def generate_completion_slow(
    model,
    enc=None,
    prefix="Hello, I'm a language model,",
    attn_mask: jax.Array | None = None,
    num_completions=8,
    max_length=20,
    temperature=0.2,
    key=None,
):
    if enc is None:
        enc = tiktoken.get_encoding("gpt2")
    if key is None:
        key = jax.random.PRNGKey(1337)

    tokens = enc.encode(prefix)
    tokens = jnp.array(tokens, dtype=jnp.int32)
    tokens = jnp.expand_dims(tokens, axis=0)
    x = jnp.tile(tokens, (num_completions, 1))
    while x.shape[1] < max_length:
        x_next = _generate_step(model, x, attn_mask, temperature, key)
        x = jnp.concatenate((x, x_next), axis=1)  # (B, T+1)#
        if attn_mask:
            attn_mask = jnp.concatenate((attn_mask, 
                                        jnp.ones((attn_mask.shape[0], 1), dtype=jnp.bool)), axis=1)
    output = []
    for i in range(num_completions):
        tokens = x[i, :max_length].tolist()
        decoded = enc.decode(tokens)
        output.append(decoded)
    return output

with mesh:
    enc = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM-360M")
    completions = generate_completion_slow(m, enc=enc, num_completions=8,
                                    max_length=41,
                                    key=jax.random.key(9),
                                    temperature=0.5,
                                    prefix="An excellent way to spend an evening in New York City")
    for completion in completions:
        print(completion)
        print('---------------------------')

An excellent way to spend an evening in New York City locomotives clo pouring Bris Barrierlog CohenopeciaU Quantumsrc examiner Channelorrect Barrier Fifth Québecrogram IRS basketorable autismUUTC testosterone examinertp Fifth Québec simulate
---------------------------
An excellent way to spend an evening in New York City delivered locomotives BarrierefficientExpl Quantum fully isotope championshipopecia Kamront agitation phenomena Channel genomes squares Posted supers fullyU Harm aneurys revered fuse nucleus demonstration num modality Ong
---------------------------
An excellent way to spend an evening in New York City Pull Pull Pull Pull Pull Pull Pull Pull Pull Pull Pull Pullometimesometimesometimesometimesometimesometimesometimesometimesometimesometimesometimesometimesometimesometimesometimesometimesometimesometimes
---------------------------
An excellent way to spend an evening in New York City delivered delivered delivered delivered delivered Barrier Barrier Barrier Barrier Barr